**Configuración**

In [1]:
import cv2, time, warnings
import numpy as np
from pynq.lib.video import *
warnings.filterwarnings('ignore')
import pynq
from pynq.overlays.base import BaseOverlay
from pynq import Overlay
from pynq import allocate
import psutil

# Función para medir la CPU y la memoria utilizada
def measure_resources():
    cpu_percent = psutil.cpu_percent()
    memory_percent = psutil.virtual_memory().percent
    return cpu_percent, memory_percent

overlay = BaseOverlay('base_w_sobel.bit')

sobel = overlay.sobel_0
dma = overlay.axi_dma_0

def reset_all_dma():
    def dma_reset(dma):
        dma.sendchannel.stop()
        dma.recvchannel.stop()
        dma.sendchannel.start()
        dma.recvchannel.start()
    dma_reset(dma)
reset_all_dma()

resolution=(640,480)
padd=int((max(resolution)-min(resolution))/2)
recive_head=0+padd
recive_tail=recive_head+min(resolution)
buffer_shape=max(resolution)

input_buffer = allocate(shape=(buffer_shape,buffer_shape), dtype=np.uint8)
output_buffer = allocate(shape=(buffer_shape,buffer_shape), dtype=np.uint8)

def run_sobel():
    reset_all_dma()
    dma.sendchannel.transfer(input_buffer)
    dma.recvchannel.transfer(output_buffer)
    sobel.write(0x10,buffer_shape)
    sobel.write(0x18,buffer_shape)
    sobel.write(0x00,0x81) #start
    dma.sendchannel.wait()
    dma.recvchannel.wait()

frame_in_w,frame_in_h = resolution
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
cap.isOpened()

[ WARN:0] global ./modules/videoio/src/cap_gstreamer.cpp (616) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


True

**Software**

In [2]:
'''
Usando OpenCV (sin acelerador PL)
'''
frame_out_w,frame_out_h = resolution
Mode = VideoMode(frame_out_w,frame_out_h,24)
hdmi_out = overlay.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

start=time.time()
num_frames=0
readError=0
total_cpu_usage = 0
total_memory_usage = 0
while (overlay.buttons[3].read()==0):
    fstime=time.time()
    ret, frame_vga = cap.read()
    if (ret):
        outframe = hdmi_out.newframe()
        laplacian_frame = cv2.Laplacian(frame_vga, cv2.CV_8U,ksize=3, dst=outframe)
        cv2.putText(outframe,"FPS:"+str(round(1/(time.time()-fstime),4)),(10,20),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(outframe)
        num_frames+=1
        # Medir recursos después de procesar el frame
        cpu_usage, memory_usage = measure_resources()
        total_cpu_usage += cpu_usage
        total_memory_usage += memory_usage
    else:
        readError += 1
        print("Error al leer de la cámara.")
        
end = time.time()
print("Prueba finalizada")
print("Frames por segundo: " + str((num_frames-readError) / (end - start)))
print("Uso promedio de CPU: " + str(total_cpu_usage / num_frames) + "%")
print("Uso promedio de Memoria: " + str(total_memory_usage / num_frames) + "%")
print("Número de errores leídos: " + str(readError))


Break signal...
Frames per second: 9.608195171580993
Average CPU Usage: 57.338926174496635%
Average Memory Usage: 64.55167785234923%
Number of read errors: 0


**Hardware**

In [3]:
'''
Con acelerador PL
'''
frame_out_w,frame_out_h = resolution
Mode = VideoMode(frame_out_w,frame_out_h,8)
hdmi_out = overlay.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_GRAY)
hdmi_out.start()

start=time.time()
num_frames=0
readError=0
total_cpu_usage = 0
total_memory_usage = 0
while (overlay.buttons[3].read()==0):
    fstime=time.time()
    frame, image=cap.read()
    if (frame):
        #image=cv2.GaussianBlur(image,(3,3),0)
        image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        image= cv2.copyMakeBorder(image,padd,padd,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
        input_buffer[:,:] = image
        run_sobel()
        outframe=hdmi_out.newframe()
        outframe=output_buffer[recive_head:recive_tail,:]
        cv2.putText(outframe,"FPS:"+str(round(1/(time.time()-fstime),4)),(1,20),0,0.8,(255,255,255),1)
        hdmi_out.writeframe(outframe)
        num_frames+=1
        # Medir recursos después de procesar el frame
        cpu_usage, memory_usage = measure_resources()
        total_cpu_usage += cpu_usage
        total_memory_usage += memory_usage
    else:
        readError+=1
        print("Error al leer de la cámara.")
        
end=time.time()
print("Prueba finalizada")
print("Frames por segundo: " + str((num_frames-readError) / (end - start)))
print("Uso promedio de CPU: " + str(total_cpu_usage / num_frames) + "%")
print("Uso promedio de Memoria: " + str(total_memory_usage / num_frames) + "%")
print("Número de errores leídos: " + str(readError))

Break signal...
Frames per second: 12.68363272748899
Average CPU Usage: 35.70733333333336%
Average Memory Usage: 64.21999999999983%
Number of read errors: 0
